# Previsão de Odds em Jogos de Futebol Europeu

***
Tema 4C, 27/05/2020

* João Alberto Preto Rodrigues Praça, up201704748@fe.up.pt 
* Liliana Natacha Nogueira de Almeida, up201706908@fe.up.pt	
* Silvia Jorge Moreira da Rocha, up201704684@fe.up.pt

Faculdade de Engenharia da Universidade do Porto <br/>
Mestrado Integrado em Engenharia Informática e Computação, Inteligência Artificial


#### Resumo

Neste artigo são abordados diferentes métodos de aprendizagem computacional, mais especificamente de aprendizagem supervisionada, na resolução de problemas de regressão. O problema em particular abordado é o de cálculo de odds para jogos, de futebol europeu, tendo por base o dataset fornecido. Existe uma fase inicial de pré-processamento dos dados (que engloba a limpeza e exploração dos dataset fornecido), seguida pela aplicação dos seguintes algoritmos: K-nearest neighbour, decision trees, neural networks e support vector machines.
....................................................(resultados e conclusões)


### 1. Introdução

O objetivo deste projeto é resolver um problema de regressão, nomeadamente a previsão de odds em jogos de futebol europeu, através da aplicação de diferentes métodos de Machine Learning, mais especificamente de aprendizagem supervisionada. 

Machine Learning é uma área da inteligência artificial que permite a um sistema computacional aprender através do reconhecimento de padrões, melhorando progressivamente a performance numa tarefa específica, sem ser programada especificamente para o efeito. 

Neste artigo iremos focar-nos na aprendizagem supervisionada que se baseia no mapeamento de inputs em outputs através do treino de um modelo. Deste modo, são fornecidos dados ao sistema, juntamente com o output desejado, de forma a ensiná-lo e ajudá-lo na identificação de padrões. Esta fase de treino irá permitir a criação de um modelo capaz de prever outputs para novos inputs daquele tipo.

O problema abordado neste documento é de regressão dado que a variável a prever (odds) é um valor real.

As odds de um jogo de futebol são cotações dadas a um determinado jogo, ou seja, definem a probabilidade de um determinado evento ocorrer. Tipicamente, as odds são calculadas por especialistas e são previstas tendo por base diversos dados estatísticos como: informações sobre a equipa, dados sobre os jogos ou eventos.  


### 2. Descrição do problema e dataset

Este dataset tem informações básicas sobre o jogo (qual o país, época, liga, jornada, data, os jogadores em campo, as suas posições, número de golos), estatísticas da FIFA acerca dos jogadores (um rating geral, o potencial, altura, peso, entre outros) e das equipas (build-up através de passes, build-up através de dribles, pressão defensiva, pressão ofensiva, criação de passes e remates, entre outros ) e dados de odds provenientes de diversas casas de apostas.

O problema de regressão em causa consiste na previsão de odds para os três cenários possíveis de um determinado jogo - vitória da equipa da casa, empate ou vitória da equipa de fora. 


### 3. Approach 

Tendo em conta a constituição do dataset, já descrita anteriormente, foram seguidos os seguintes passos na fase de seleção de dados:

* Na tabela relativa aos atributos de uma dada equipa, decidimos compilar todos os dados fornecidos através da média ponderada dos valores de build-up através de velocidade, build-up através de drible, build-up através de passes, pressão defensiva, pressão ofensiva e largura da defesa. 
* Na tabela relativa aos atributos de um jogador não foram utilizados nenhuns dados uma vez que o dataset é composto por informação proveniente da FIFA e, portanto, a pontuação da equipa num dado momento é uma média ponderada dos seus jogadores. Deste modo, a utilização destes dados iria acrescentar complexidade e redundância sem melhorar os resultados obtidos.
* Na tabela relativa aos jogos, foram utilizados os golos da equipa da casa, da equipa de fora e uma média ponderada das odds das casas de apostas. Nesta média, foram excluídas algumas casas que apresentavam uma elevada percentagem de dados com valores omissos.
* As restantes tabelas foram apenas utilizadas para unir estas informações dado que não apresentavam mais nenhum dado relevante.

De seguida, efetuámos o pré-processamento destes dados. Nesta fase, os dados omissos foram uniformizados e, de seguida, foram removidas todas as linhas que os incluíam. A opção de eliminar linhas com dados omissos ao invés de os substituir por outros valores (como por exemplo a média de valores da coluna) assenta em dois motivos: a elevada dimensão do dataset permite que a perda de alguns dados não se demonstre relevante e a aplicação de operações usando todos os dados válidos de uma coluna (como por exemplo a de odds) não permitiria a obtenção de um valor próximo daquele que seria o valor real, uma vez que para cada jogo as equipas participantes são diferentes e portanto os dados dessa coluna não têm relação entre si.

Após isto, surge a fase de transformação. Nesta fase foi aplicada a normalização sobre os valores das variáveis a serem utilizadas pelos algoritmos de cálculo de regressão.

................................. falar do PCA 

Finalmente, surge a fase de aplicação dos algoritmos já mencionados. 



In [16]:
import numpy as np # linear algebra
import pandas as pd # data processing
import sqlite3
from sklearn import preprocessing
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor # Nearest Neighbors Algorithm
from sklearn.neural_network import MLPRegressor # Neural network Algorithm
from sklearn import svm # Support Vector Machines Algorithm
from sklearn import tree # Decision Trees Algorithm
from statsmodels.graphics.correlation import plot_corr

In [2]:
con = sqlite3.connect("./database.sqlite")
display_graph = False

def clean_data(data):
    #this line displays the NaN distribution before data clean up
    if display_graph:
        msno.matrix(data)
    data = data.replace("", np.NaN)
    data = data.dropna()
    if display_graph:
        msno.matrix(data)
    return data

def preprocess_data() :
    matches = pd.read_sql_query('SELECT *, strftime("%Y",date) as year, (B365H + BWH + IWH + LBH + WHH +  VCH )/6 AS home_odds, (B365D + BWD + IWD + LBD + WHD + VCD)/6 AS draw_odds,  (B365A + BWA + IWA + LBA + WHA + VCA )/6 AS away_odds FROM match',con)
    matches = matches.iloc[:, np.r_[0,2,4,7,8,9,10, 55:77, 115:119 ]]

    teams = pd.read_sql_query('SELECT team_api_id, (buildUpPlaySpeed + buildUpPlayDribbling + buildUpPlayPassing + defencePressure + defenceAggression + defenceTeamWidth)/6 as team_rating FROM team_attributes', con)
    players = pd.read_sql_query('SELECT player_api_id, strftime("%Y",date) as year, avg(potential) as player_potential, avg(overall_rating) as rating FROM player_attributes GROUP BY player_api_id, year', con)
    
    matches = clean_data(matches)
    teams = clean_data(teams)
    players = clean_data(players)
      
    result =pd.merge(matches, teams, left_on='home_team_api_id', right_on='team_api_id', how='left').drop('team_api_id', axis=1)
    result =pd.merge(result, teams, left_on='away_team_api_id', right_on='team_api_id', how='left').drop('team_api_id', axis=1)
    result.rename(columns={"team_rating_x": "home_team_rating", "team_rating_y": "away_team_rating"}, inplace=True)

    for index in range(1,12):
        player_home_id = "home_player_" + str(index)
        player_away_id = "away_player_" + str(index)
        result =pd.merge(result, players, left_on=[player_home_id, 'year'], right_on=['player_api_id', 'year'], how='left').drop(['player_api_id', player_home_id], axis=1)
        result =pd.merge(result, players, left_on=[player_away_id, 'year'], right_on=['player_api_id', 'year'], how='left').drop(['player_api_id', player_away_id], axis=1)
        result.rename(columns={"player_potential_x": player_home_id + "_potential", "player_potential_y": player_away_id + "_potential" , "rating_x": player_home_id + "_rating", "rating_y": player_away_id + "_rating"}, inplace=True)
    
    result = clean_data(result)
    return result


In [29]:
def PCA_call(X, Y, dataset): 
    pca = PCA(n_components=2)
        
    data_scaled = pd.DataFrame(preprocessing.scale(X),columns = X.columns) 

    principalComponents = pca.fit_transform(data_scaled)
    
   # print(pca.explained_variance_ratio_)
   # print(pca.singular_values_)
    
   # print(principalComponents)
   # principalDf = pd.DataFrame(data = principalComponents
    #         , columns = ['principal component 1', 'principal component 2'])
    
    #finalDf = pd.concat([principalDf, Y], axis = 1)
#    display(finalDf)

    print (pd.DataFrame(pca.components_,columns=data_scaled.columns, index = ['PC-1','PC-2']))

In [3]:
def linear_regression(X_train, Y_train, X_test, Y_test):
    reg = LinearRegression().fit(X_train, Y_train)

    y_pred = reg.predict(X_test)
    print(r2_score(Y_test, y_pred))

    
    #plt.scatter(X_test, Y_test,  color='gray')
    plt.plot(X_test, y_pred, color='red', linewidth=2)
    plt.plot(X_test, Y_test, color='gray', linewidth=2)
    plt.show()
    
    #print(reg.intercept_)
    #print(reg.coef_)
    


#### K Nearest Neighbours

Este algoritmo identifica quais os k vizinhos mais próximos, isto é, os k pontos do conjunto de treino que estão a uma menor distância do valor que está a ser previsto. A previsão irá corresponder à média ponderada entre esses k pontos.

In [5]:
def k_nearest_neighbour(X_train, X_test, Y_train, Y_test):
    neighbour = KNeighborsRegressor(n_neighbors=2)
    neighbour.fit(X_train, Y_train)

    neighbour_pred = neighbour.predict(X_test)
    
    print(r2_score(Y_test, neighbour_pred))

    
    #plt.scatter(X_test, Y_test,  color='gray')
    plt.plot(X_test, neighbour_pred, color='red', linewidth=2)
    plt.plot(X_test, Y_test, color='gray', linewidth=2)
    plt.show()
    


#### Neural Networks

As redes neuronais artificiais são inspiradas no funcionamento do cérebro humano. Estas redes são compostas por um elevado número de neurónios altamente ligados e a trabalhar em paralelo na resolução de problemas. Todo o seu conhecimento é adquirido através da aprendizagem e armazenado nas ligações. 

Nesta implementação é usado um Multi-Layer Perceptron. Neste tipo de modelos existe uma primeira layer (input layer), hidden layers (número variável) e uma última layer que corresponde à output layer.


In [6]:
def neural_network(X_train, X_test, Y_train, Y_test):
    neural = MLPRegressor()
    neural.fit(X_train, Y_train)

    neural_pred = neural.predict(X_test)
    print(r2_score(Y_test, neural_pred))

    
    #plt.scatter(X_test, Y_test,  color='gray')
    plt.plot(X_test, neural_pred, color='red', linewidth=2)
    plt.plot(X_test, Y_test, color='gray', linewidth=2)
    plt.show()

#### Support Vector Machines

Este algoritmo procura identificar uma função f(X) que não se poderá desviar mais do que o valor de epsilon para cada valor de y presente no conjunto de treino. Simultaneamente, esta função deve tentar aproximar-se, o mais possível, de uma reta.

In [7]:
def support_vector_machines(X_train, X_test, Y_train, Y_test):
    #epsilon default value is 0.1
    svm_instance = svm.SVR()
    svm_instance.fit(X_train, Y_train)

    svm_pred = svm_instance.predict(X_test)
    print(r2_score(Y_test, svm_pred))

    
    #plt.scatter(X_test, Y_test,  color='gray')
    plt.plot(X_test, svm_pred, color='red', linewidth=2)
    plt.plot(X_test, Y_test, color='gray', linewidth=2)
    plt.show()

#### Decision Trees

As árvores de decisão procuram ajustar-se a uma curva sinusoidal, com ligeiros desvios. Como resultado, são obtidas regressões lineares locais que aproximam essa curva.

Para utilizar árvores de decisão em problemas de regressão, temos de definir uma métrica de impureza que se adeque a variáveis contínuas. Deste modo, deve ser definida como o erro quadrático médio dos nós.


In [14]:
def decision_tree(X_train, X_test, Y_train, Y_test):
    dta = tree.DecisionTreeRegressor(criterion='mse')
    dta.fit(X_train, Y_train)
    
    dta_pred = dta.predict(X_test)
    print(r2_score(Y_test, dta_pred))

   # plt.scatter(X_test, Y_test,  color='gray')
    plt.plot(X_test, dta_pred, color='red', linewidth=2)
    plt.plot(X_test, Y_test, color='gray', linewidth=2)
    plt.show()
    

In [9]:
def get_team_info(team_id, year, dataset):
    home_matches = dataset[dataset.home_team_api_id == team_id ] #('(home_team_api_id == @team_id) & (year == @year)')
    away_matches = dataset[dataset.away_team_api_id == team_id ] #('(home_team_api_id == @team_id) & (year == @year)')
    team_matches = away_matches.append(home_matches)
    team_matches = team_matches[team_matches.year == year ]
    return team_matches 

In [30]:
#def calculate_odds(match_id):
#match = pd.read_sql_query('SELECT * FROM match WHERE id = {}'.format(1), con)
dataset = preprocess_data()
#print(dataset)

team_dataset = get_team_info(9984, '2009', dataset)

X = team_dataset.iloc[:, [5,6,11, 12]]
print(X)
X1 = preprocessing.scale(X)

Y = team_dataset.iloc[:, 8]
Y = preprocessing.scale(Y)

dba = team_dataset.iloc[:, [5,6,8,9,10,11,12]]

corr = dba[:-1].corr()

#fig=plot_corr(corr,xnames=corr.columns)

X_train,X_test,Y_train,Y_test = train_test_split(X1,Y,test_size= 0.3)

#linear_regression(X_train, Y_train, X_test, Y_test)
PCA_call(X, Y, dataset)
#k_nearest_neighbour(X_train, X_test, Y_train, Y_test)
#neural_network(X_train, X_test, Y_train, Y_test)
#support_vector_machines(X_train, X_test, Y_train, Y_test)
#decision_tree(X_train, X_test, Y_train, Y_test)

     home_team_goal  away_team_goal  home_team_rating  away_team_rating
88                1               1              51.0              52.0
89                1               1              51.0              52.0
90                1               1              51.0              52.0
91                1               1              51.0              52.0
140               3               1              51.0              52.0
141               3               1              51.0              52.0
142               3               1              49.0              52.0
143               3               1              49.0              52.0
155               1               0              47.0              52.0
156               1               0              47.0              52.0
157               1               0              47.0              52.0
158               1               0              47.0              52.0
175               3               2              54.0           

### 4. Experimental evaluation


### 5. Conclusions


### References